In [ ]:
%run -i setup.py

# Time Series Test

Here we are taking three numeric series with datestamps. We'll aggregate by day, then plot each line on the same figure.

In [ ]:
import matplotlib.dates as mdates
from datetime import datetime
dataset['release_date'] = [datetime.strptime(d, "%Y-%m-%d") if len(d) > 9 else 'NaN' for d in dataset['release_date']]
sample = dataset[dataset['release_date']!='NaN']

In [ ]:
grouped_sample = sample.groupby('release_date', as_index=False).agg({"acousticness": "mean",
 "danceability": [min, max, 'mean', 'std'],
  "duration_ms": [min, max, 'mean', 'std'],
   "energy": [min, max, 'mean', 'std'],
    "explicit": [min, max, 'mean', 'std'],
    "instrumentalness": [min, max, 'mean', 'std'],
    "loudness": [min, max, 'mean', 'std'],
    "popularity": [min, max, 'mean', 'std'],
    "tempo": [min, max, 'mean', 'std'],
    "valence": [min, max, 'mean', 'std']})

grouped_sample.columns = [' '.join(col).strip() for col in grouped_sample.columns.values]

### Matplotlib

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set(title = "Danceability by Release Date")
b1 = ax.plot(grouped_sample['release_date'], grouped_sample['danceability mean'],color = 'blue', alpha = .75)
b2 = ax.plot(grouped_sample['release_date'], grouped_sample['danceability min'],color = 'red', alpha = .25)
b3 = ax.plot(grouped_sample['release_date'], grouped_sample['danceability max'],color = 'green', alpha = .25)

ax.set_xlabel("Release Date")
ax.set_ylabel("Danceability")

### Seaborn

To access all three series, the data needs to be melted long - a really annoying task.

In [ ]:
sample2 = grouped_sample[['release_date', 'danceability mean', 'danceability min', 'danceability max']]

f, ax = plt.subplots(figsize=(12, 6))

with sns.axes_style("whitegrid"):
    g = sns.lineplot(
        ax=ax,
        data=pd.melt(sample2, ['release_date']), 
        x="release_date", 
        y="value", 
        hue='variable',
        alpha = .5
    )
    g.set_title("Danceability by Release Date")
    g.set_xlabel("Release Date")
    g.set_ylabel("Danceability")
    g

### Bokeh

In [ ]:
output_notebook()

p = figure(title="Danceability by Release Date", 
           y_axis_label='Danceability', 
           x_axis_label='Release Date', 
           width=700, 
           x_axis_type='datetime',
           height = 400)

p.line(x=grouped_sample['release_date'], 
    y=grouped_sample['danceability mean'], color = 'blue', alpha = .75)

p.line(x=grouped_sample['release_date'], 
    y=grouped_sample['danceability min'], color = 'red', alpha = .25)

p.line(x=grouped_sample['release_date'], 
    y=grouped_sample['danceability max'], color = 'green',alpha = .25)

show(p)

### Altair

Again we need to melt the data to get the desired effect, but this means the limitations of Altair are much more problematic, as we have even less data than usual.

In [ ]:
sample2 = grouped_sample[['release_date', 'danceability mean', 'danceability min', 'danceability max']]
sample2 = pd.melt(sample2, ['release_date'])
source = sample2.sample(axis = 0, n=4000)

viz = alt.Chart(source)
viz = viz.mark_line()
viz = viz.encode(
    x='release_date:T', # Date must be cast to T type- had to research this
    y='value:Q',
    color=alt.Color('variable', scale=alt.Scale(scheme='set1'))
)
viz = viz.configure_mark(opacity=0.5)
viz = viz.properties(title='Danceability by Release Date').properties(width=600, height=300)

viz

### Plotnine

In [ ]:
from mizani.breaks import date_breaks
from mizani.formatters import date_format

pno.dpi = (100)
pno.figure_size = (8,4)
ggplot(data=grouped_sample) + \
    theme_bw() + \
    geom_line(aes(x='release_date', y='danceability mean'), color = 'blue', alpha = .75) + \
    geom_line(aes(x='release_date', y='danceability min'), color = 'red', alpha = .25) + \
    geom_line(aes(x='release_date', y='danceability max'), color = 'green', alpha = .25) + \
    scale_x_datetime(breaks=date_breaks('10 years'), labels=date_format('%Y')) +\
    labs(title = "Danceability by Release Date", x="Release Date", y="Danceability")

### Plotly

We can use melted data and stick to Plotly Express, or go with the more extensive option and add traces. Plotly Express cannot do opacity for lines.

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=grouped_sample['release_date'], y=grouped_sample['danceability mean'],
                    mode='lines', opacity=.75,
                    name='danceability mean'))
fig.add_trace(go.Scatter(x=grouped_sample['release_date'], y=grouped_sample['danceability min'],
                    mode='lines',opacity=.25,
                    name='danceability min'))
fig.add_trace(go.Scatter(x=grouped_sample['release_date'], y=grouped_sample['danceability max'],
                    mode='lines', opacity=.25,name='danceability max'))
fig.update_layout(
    title = "Danceability by Release Date",
    width=700,height=400,
    template='plotly_white',
    margin=dict(l=15,r=25,b=15,t=40,pad=1))
fig.show()

## Time Series Results

Unscientific rankings by me, 1 = best, 6 = worst. Notable here: some libraries are much better at handling dates than others. Requiring melt of data reduces features score - other libraries accept each line series separately to be specifically defined (eg Traces in Plotly). Without defining each line series separately, it becomes very hard to customize opacity or color per series.

|            |features     |beauty    |ease of use   |
|------------|-------------|----------|--------|
|matplotlib  |    4        |    5     |   1   |
|seaborn     |    6        |    4     |   3   |
|bokeh       |    2        |    1     |   2   |
|altair      |    5        |    2     |   4   |
|plotnine    |    3        |    6     |   6   |
|plotly      |    1        |    3     |   5   |

